In [1]:
# This project aims to predict the sale price of Bulldozers using Machine Learning Regression models
# Note: Kaggle has set the evaluation metric to being root mean squared log error (RMSLE).

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [131]:
# Importing data
df= pd.read_csv('bulldozers/TrainAndValid.csv',low_memory=False,parse_dates=['saledate'])

In [132]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412698 entries, 0 to 412697
Data columns (total 53 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   SalesID                   412698 non-null  int64         
 1   SalePrice                 412698 non-null  float64       
 2   MachineID                 412698 non-null  int64         
 3   ModelID                   412698 non-null  int64         
 4   datasource                412698 non-null  int64         
 5   auctioneerID              392562 non-null  float64       
 6   YearMade                  412698 non-null  int64         
 7   MachineHoursCurrentMeter  147504 non-null  float64       
 8   UsageBand                 73670 non-null   object        
 9   saledate                  412698 non-null  datetime64[ns]
 10  fiModelDesc               412698 non-null  object        
 11  fiBaseModel               412698 non-null  object        
 12  fi

In [133]:
# Sort dataframe by date 
df.sort_values(by=["saledate"], inplace=True, ascending=True)

In [134]:
# Added datetime parameters for saledate
def preprocess_date(df):
    df["saleYear"] = df.saledate.dt.year
    df["saleMonth"] = df.saledate.dt.month
    df["saleDay"] = df.saledate.dt.day
    df["saleDayofweek"] = df.saledate.dt.dayofweek
    df["saleDayofyear"] = df.saledate.dt.dayofyear

    # Dropped original saledate
    df.drop("saledate", axis=1, inplace=True)
    return df
preprocess_date(df)

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,saleYear,saleMonth,saleDay,saleDayofweek,saleDayofyear
205615,1646770,9500.0,1126363,8434,132,18.0,1974,NaN,NaN,TD20,...,None or Unspecified,Straight,None or Unspecified,NaN,NaN,1989,1,17,1,17
274835,1821514,14000.0,1194089,10150,132,99.0,1980,NaN,NaN,A66,...,NaN,NaN,NaN,Standard,Conventional,1989,1,31,1,31
141296,1505138,50000.0,1473654,4139,132,99.0,1978,NaN,NaN,D7G,...,None or Unspecified,Straight,None or Unspecified,NaN,NaN,1989,1,31,1,31
212552,1671174,16000.0,1327630,8591,132,99.0,1980,NaN,NaN,A62,...,NaN,NaN,NaN,Standard,Conventional,1989,1,31,1,31
62755,1329056,22000.0,1336053,4089,132,99.0,1984,NaN,NaN,D3B,...,None or Unspecified,PAT,Lever,NaN,NaN,1989,1,31,1,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410879,6302984,16000.0,1915521,5266,149,99.0,2001,NaN,NaN,D38E,...,None or Unspecified,PAT,None or Unspecified,NaN,NaN,2012,4,28,5,119
412476,6324811,6000.0,1919104,19330,149,99.0,2004,NaN,NaN,2064,...,NaN,NaN,NaN,NaN,NaN,2012,4,28,5,119
411927,6313029,16000.0,1918416,17244,149,99.0,2004,NaN,NaN,337G,...,NaN,NaN,NaN,NaN,NaN,2012,4,28,5,119
407124,6266251,55000.0,509560,3357,149,99.0,1993,NaN,NaN,12G,...,NaN,NaN,NaN,NaN,NaN,2012,4,28,5,119


In [135]:
# Checking for missing values in dataframe
df.isna().sum()

SalesID                          0
SalePrice                        0
MachineID                        0
ModelID                          0
datasource                       0
auctioneerID                 20136
YearMade                         0
MachineHoursCurrentMeter    265194
UsageBand                   339028
fiModelDesc                      0
fiBaseModel                      0
fiSecondaryDesc             140727
fiModelSeries               354031
fiModelDescriptor           337882
ProductSize                 216605
fiProductClassDesc               0
state                            0
ProductGroup                     0
ProductGroupDesc                 0
Drive_System                305611
Enclosure                      334
Forks                       214983
Pad_Type                    331602
Ride_Control                259970
Stick                       331602
Transmission                224691
Turbocharged                331602
Blade_Extension             386715
Blade_Width         

In [136]:
# Reusable function for data preprocessing
def preprocessing_data(df):
    for label, content in df.items():
        if pd.api.types.is_string_dtype(content):
            df[label]=content.astype('category').cat.as_ordered()
        
    for label,content in df.items():
        if pd.api.types.is_numeric_dtype(content):
            if pd.isnull(content).sum():
                df[label]= content.fillna(content.median())
    for label,content in df.items():
        if  pd.api.types.is_categorical_dtype(content):
#         we add +1 since pandas encodes missing values as -1
            df[label]=pd.Categorical(content).codes+1
    return df

In [139]:
df=preprocessing_data(df)

# Checking if there is still columns having string data type
for label, content in df.items():
    if pd.api.types.is_string_dtype(content):
        print(label)

In [140]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 412698 entries, 205615 to 409203
Data columns (total 57 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SalesID                   412698 non-null  int64  
 1   SalePrice                 412698 non-null  float64
 2   MachineID                 412698 non-null  int64  
 3   ModelID                   412698 non-null  int64  
 4   datasource                412698 non-null  int64  
 5   auctioneerID              412698 non-null  float64
 6   YearMade                  412698 non-null  int64  
 7   MachineHoursCurrentMeter  412698 non-null  float64
 8   UsageBand                 412698 non-null  int8   
 9   fiModelDesc               412698 non-null  int16  
 10  fiBaseModel               412698 non-null  int16  
 11  fiSecondaryDesc           412698 non-null  int16  
 12  fiModelSeries             412698 non-null  int8   
 13  fiModelDescriptor         412698 non-nu

In [142]:
# Checking if there are still columns of numeric data type contain null values
for label,content in df.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)

In [144]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 412698 entries, 205615 to 409203
Data columns (total 57 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SalesID                   412698 non-null  int64  
 1   SalePrice                 412698 non-null  float64
 2   MachineID                 412698 non-null  int64  
 3   ModelID                   412698 non-null  int64  
 4   datasource                412698 non-null  int64  
 5   auctioneerID              412698 non-null  float64
 6   YearMade                  412698 non-null  int64  
 7   MachineHoursCurrentMeter  412698 non-null  float64
 8   UsageBand                 412698 non-null  int8   
 9   fiModelDesc               412698 non-null  int16  
 10  fiBaseModel               412698 non-null  int16  
 11  fiSecondaryDesc           412698 non-null  int16  
 12  fiModelSeries             412698 non-null  int8   
 13  fiModelDescriptor         412698 non-nu

In [145]:
df.saleYear.value_counts()

2009    43849
2008    39767
2011    35197
2010    33390
2007    32208
2006    21685
2005    20463
2004    19879
2001    17594
2000    17415
2002    17246
2003    15254
1998    13046
1999    12793
2012    11573
1997     9785
1996     8829
1995     8530
1994     7929
1993     6303
1992     5519
1991     5109
1989     4806
1990     4529
Name: saleYear, dtype: int64

In [146]:
# According to the Kaggle data page, the validation set and test set are split according to dates
df_train = df[df.saleYear != 2012]
df_val = df[df.saleYear ==2012]
len(df_train), len(df_val)

(401125, 11573)

In [147]:
# Splitting the data in training and validation sets
X_train,y_train = df_train.drop('SalePrice',axis=1), df_train.SalePrice
X_val,y_val = df_val.drop('SalePrice',axis=1), df_val.SalePrice


X_train.shape, y_train.shape, X_val.shape, y_val.shape

((401125, 56), (401125,), (11573, 56), (11573,))

In [112]:
# With sorted data lets build model

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

model_rf = RandomForestRegressor(n_jobs=-1,max_samples=10000)
model_lr=LinearRegression(n_jobs=-1)

In [113]:
model_rf.fit(X_train,y_train)

RandomForestRegressor(max_samples=10000, n_jobs=-1)

In [114]:
model_lr.fit(X_train,y_train)

LinearRegression(n_jobs=-1)

In [117]:
# Created evaluation function Root Mean Square Log Error
from sklearn.metrics import mean_absolute_error,mean_squared_log_error

def rmsle (y_test,y_preds):
    return np.sqrt(mean_squared_log_error(y_test, y_preds))

def print_scores(model):
    train_preds= model.predict(X_train)
    val_preds= model.predict(X_val)
    scores = {"Training MAE": mean_absolute_error(y_train, train_preds),
              "Valid MAE": mean_absolute_error(y_val, val_preds),
              "Training RMSLE": rmsle(y_train, train_preds),
              "Valid RMSLE": rmsle(y_val, val_preds),
              "Training R^2": model.score(X_train, y_train),
              "Valid R^2": model.score(X_val, y_val)}
    return scores

In [118]:
print_scores(model_rf)

{'Training MAE': 5556.832880124651,
 'Valid MAE': 7153.982936144475,
 'Training RMSLE': 0.25767944108112767,
 'Valid RMSLE': 0.2927325221709245,
 'Training R^2': 0.8608528630042187,
 'Valid R^2': 0.8340396275101308}

In [123]:
# Lets try to tune Hyperparameters with RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV

rf_grid={'n_estimators':np.arange(10,100,10),
        'max_depth':[None,3,5,10],
        'min_samples_split':np.arange(2,20,2),
        'min_samples_leaf':np.arange(1,20,2),
        'max_features':[0.5,1,'sqrt','auto'],
        'max_samples':[10000]}

rs_rf= RandomizedSearchCV(RandomForestRegressor(),rf_grid,n_iter=20,cv=5,verbose=True)

rs_rf.fit(X_train,y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  5.4min finished


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=20,
                   param_distributions={'max_depth': [None, 3, 5, 10],
                                        'max_features': [0.5, 1, 'sqrt',
                                                         'auto'],
                                        'max_samples': [10000],
                                        'min_samples_leaf': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]),
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18]),
                                        'n_estimators': array([10, 20, 30, 40, 50, 60, 70, 80, 90])},
                   verbose=True)

In [124]:
rs_rf.best_params_

{'n_estimators': 60,
 'min_samples_split': 6,
 'min_samples_leaf': 5,
 'max_samples': 10000,
 'max_features': 0.5,
 'max_depth': None}

In [125]:
print_scores(rs_rf)

{'Training MAE': 6068.628250486682,
 'Valid MAE': 7501.92800467421,
 'Training RMSLE': 0.27575399879181484,
 'Valid RMSLE': 0.30314338069473606,
 'Training R^2': 0.8354868902081609,
 'Valid R^2': 0.8155509496375177}

In [126]:
model=RandomForestRegressor(n_estimators=60,min_samples_split=6,min_samples_leaf=5,max_samples=10000,max_features=0.5,max_depth=None)
model.fit(X_train,y_train)

RandomForestRegressor(max_features=0.5, max_samples=10000, min_samples_leaf=5,
                      min_samples_split=6, n_estimators=60)

In [127]:
print_scores(model)

{'Training MAE': 6099.759096025493,
 'Valid MAE': 7519.020280932356,
 'Training RMSLE': 0.2775072420542142,
 'Valid RMSLE': 0.30561692896845244,
 'Training R^2': 0.8339059967249387,
 'Valid R^2': 0.8162863899219154}

In [128]:
# Making predictons on test data
df_test=pd.read_csv('bulldozers/Test.csv',parse_dates=['saledate'])

In [148]:
df_test=preprocess_date(df_test)

In [149]:
df_test=preprocessing_data(df_test)

In [151]:
# Make predictions on the test dataset using the best model
test_preds=model.predict(df_test)

In [152]:
# Create DataFrame compatible with Kaggle submission requirements
df_preds = pd.DataFrame()
df_preds["SalesID"] = df_test["SalesID"]
df_preds["SalePrice"] = test_preds
df_preds

,SalesID,SalePrice
0,1227829,20619.838280
1,1227844,19290.328042
2,1227847,52408.333333
3,1227848,59506.868386
4,1227863,44629.756944
...,...,...
12452,6643171,36147.262262
12453,6643173,13234.326720
12454,6643184,12778.417659
12455,6643186,16714.818122
